In [15]:
import pandas as pd
import h5py
import re
import os
import numpy as np 

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from selenobot.files import FASTAFile

from transformers import AutoTokenizer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# tokenizer = AutoTokenizer.from_pretrained('facebook/esm2_t36_3B_UR50D', clean_up_tokenization_spaces=True)
# tokenizer.get_vocab()

df = pd.read_csv('../data/3c_xl_metadata_train.csv', index_col=0)
df[df.seq.str.contains('U')]

,mmseqs_representative,mmseqs_cluster,domain,kingdom,phylum,class,order,family,genus,species,ncbi_taxonomy_id,refseq_protein_id,refseq_nucleotide_id,non_terminal_residue,function,seq,existence,name,kegg,file_name,label,sec_index_n,sec_index_c,sec_count,trunc_size,trunc_ratio,original_length,dataset
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [6]:
# Generate FASTA files for KEGG annotation. 
fasta_files = dict()
fasta_files['uniprot_sec.faa'] = pd.read_csv('../data/uniprot_metadata_sec.csv', index_col=0)
fasta_files['uniprot_sec_bacteria.faa'] = metadata_sec_df[metadata_sec_df.domain == 'Bacteria']
seqs, index = [s[:s.index('U')] for s in metadata_sec_bac_df.seq], pd.Series(name='id', data=[id_ + '-' for id_ in metadata_sec_bac_df.index])
fasta_files['uniprot_sec_bacteria_truncated.faa'] = metadata_sec_bac_df.copy().assign(seq=seqs).set_index(index)

for file_name, df in fasta_files.items():
    path = os.path.join('../data', file_name)
    fasta_file = FASTAFile.from_df(df, add_description=False)
    fasta_file.write(path)


In [6]:
confusion_matrix = sklearn.metrics.confusion_matrix(test_metadata_df.label.values, test_metadata_df.prediction.values)

image = ax.imshow(confusion_matrix, cmap='RdBu')

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xlabel('prediction')
ax.set_ylabel('label')
for i in range(confusion_matrix.shape[0]):
    for j in range(confusion_matrix.shape[-1]):
        text = ax.text(i, j, confusion_matrix[i, j], ha='center', va='center', color='white')

image = ax.figure.colorbar(image, ax=ax)
# Most of the erroneous predictions are false positives. 
plt.show()